# Week1_Library 과제

### Q1. Library 와 Framework 의 차이 간단하게 서술하시오. (100자 내외)

Library란 컴퓨터 프로그램이 자주 사용하는 코드 및 함수의 집합으로, tensorflow나 numpy를 예로 들 수 있다. Framework란 컴퓨터 프로그램 개발에 사용되는 골격으로, Android나 Spring을 예로 들 수 있다.

### Q2. 딥러닝과 머신러닝의 관계 및 특징, 차이 간단하게 서술하시오. (200자 내외)

머신러닝은 기계가 어떤 경험적 자료를 바탕으로 그것의 성능을 개선할 수 있는 일체의 프로그램 개발 방식을 의미한다. 딥러닝은 이러한 머신러닝 방식 중 하나로, 딥러닝 알고리즘은 기본적으로 선형변환(batch dim이 1일 경우 행렬곱)을 여러 계층 합성하는 방식으로 구성된다. 딥러닝 알고리즘의 성능은 미분을 통해 오차를 감소시키는 방향으로 선형변환 파라미터의 값을 점진적으로 변화시키는 방식으로 개선된다.

### Q3. 아래의 코드에 주석 달기.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(45)
if device == 'cuda':
    torch.cuda.manual_seed_all(45)
print(device + " is available")

"""
신경망 개발에 필요한 파이토치 라이브러리를 import하였다.
만약 CUDA 기반 GPU를 쓸 수 있다면 GPU를 사용하고, 그렇지 않으면 CPU를 사용한다.
"""

In [1]:
learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5

"""
학습에 필요한 각종 파라미터 중 User가 직접 정해줘야 하는 파라미터를
위와 같이 설정하였다.
"""

In [ ]:

train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor() 
    ])
)

test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)

"""
MNIST 데이터셋을 다운받아
훈련에 필요한 train set과 검증에 필요한 test set으로 나누었다.
"""

In [ ]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

"""
위의 데이터셋을 각각 batch size 100으로 묶어
한 번에 batch size만큼 load한다.
"""

In [ ]:

class ConvNet(nn.Module):
  def __init__(self): 
        super(ConvNet, self).__init__() 
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) 
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) 
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) 
        self.mp = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(320,100) 
        self.fc2 = nn.Linear(100,10) 


  def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) 
        x = F.relu(self.mp(self.conv2(x))) 
        x = self.drop2D(x) 
        x = x.view(x.size(0), -1) 
        x = self.fc1(x)
        x = self.fc2(x)
        return F.log_softmax(x)
  
"""
CNN 기반 네트워크를 만들고, 각 층마다 호출될 Forward 함수를 설계했다.
해당 네트워크는 Conv -> ReLU -> Conv -> ReLU -> Dropout -> MaxPooling -> Linear -> Linear -> Softmax 
의 결과를 다음 계층으로 전달한다.
"""

In [ ]:

model = ConvNet().to(device) 
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
"""
위에서 만든 모델이 전파한 값에 대한 손실함수는 크로스엔트로피 함수로 설정하며,
Optimizer는 가장 빈번히 쓰이는 Adam Optimizer이다.
"""

In [ ]:

for epoch in range(epochs): 
    avg_cost = 0
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() 
        hypothesis = model(data)
        cost = criterion(hypothesis, target) 
        cost.backward()
        optimizer.step() 
        avg_cost += cost / len(train_loader) 
    print('[Epoch: {:>4}]  cost = {:>.9}'.format(epoch + 1, avg_cost))

"""
5에포크동안
Train 데이터와 정답값을 받아
Model에 데이터를 넣어 Hypothesis와 Target 사이의 Loss를 계산한다.
각 에포크동안 평균 Loss를 출력한다.
"""

In [ ]:

model.eval()
with torch.no_grad(): 
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] 
        total += len(target) 
        correct += (preds==target).sum().item() 
        
    print('Test Accuracy: ', 100.*correct/total, '%')

"""
Test 데이터와 정답값을 바탕으로 검증한 뒤,
평균정답률(Accuracy)를 산출한다.
"""
     

## 첫 정규세션 들으시느라 고생 많으셨습니다.